## Funciones y librerias

In [ ]:
import speech_recognition as sr                                                                                         # Importo el módulo 'speech_recognition' y le asigno el alias 'sr'. (Este módulo me permite convertir el habla en texto, lo que es útil para aplicaciones de reconocimiento de voz.)
import serial                                                                                                           # Importo el módulo 'serial', que me permite comunicarme con puertos seriales.
from pynput import keyboard                                                                                             # Importo el módulo 'keyboard' del paquete 'pynput' para manejar eventos del teclado.(Esto me permite detectar y reaccionar a las teclas que se presionan o se liberan en el teclado)

########################################################################################################################################################
########################################################################################################################################################
#                                         FUNCIONES PARA USAR EN EL MANEJO DEL BRAZO CON LA VOZ                                                        #
########################################################################################################################################################
########################################################################################################################################################

def crear_diccionario_correcciones():                                                                                   # Funcion para las palabras complicadas de recibir , y evitar posibles fallas de palabras mal interpretadas.
    return {
        "UNO": "1", "DOS": "2", "TRES": "3", "CUATRO": "4",
        "CINCO": "5", "SEIS": "6", "SIETE": "7", "OCHO": "8",
        "BE": "B", "VE": "B", "CE": "C", "DE ": "D", "DE": "D",
        "ES": "E", "ES ": "E", "EFE": "F", "GE": "G", "ACHE": "H", "HACHE": "H",
        "X": "*", "POR": "*",
        "A 6": "A6", "A SEIS": "A6",
    }

# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#

def Presionar_tecla(key):                                                                                               # Defino la función 'Presionar_tecla' que toma un parámetro llamado 'key'.(Esa parametro es la tecla que se presiono)
    global continuar
    if hasattr(key, 'char'):                                                                                            # Verifico si el objeto 'key' tiene el atributo 'char'.
        if key.char == 'h':                                                                                             # Si la tecla presionada es 'h', la función devuelve 'False'
            return False
        elif key.char == 'q':                                                                                           # Sino...si la tecla presionada es 'q', realizo las siguientes acciones:
            continuar = False                                                                                           # Esto lo utilizo para detener el bucle principal                                                                                       
            return False

# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#

def Reconocer_coordenadas():                                                                                            # Defino la función 'Reconocer_coordenadas' que no toma ningún parámetro.
    correcciones = crear_diccionario_correcciones()                                                                     # Basicamente coloco una variable llamada correcciones que tiene un diccionario de correciones, para las posibles palabras con las que puede fallar el audio.
    recognizer = sr.Recognizer()                                                                                        # Esto me permite usar el reconocimiento de voz para convertir audio en texto.
    print("Presiona 'H' para comenzar a hablar...")                                                                     # Imprimo un mensaje en la consola para indicar que el programa está esperando que se presione 'H' para comenzar.

    with keyboard.Listener(on_press=Presionar_tecla) as listener:                                                       # Inicia un escucha de eventos del teclado que usa la función 'Presionar_tecla' para manejar las teclas presionadas. (El 'listener' estará activo hasta que la función 'Presionar_tecla' devuelva 'False')
        listener.join()
    
    if not continuar:                                                                                                   # Si la variable global 'continuar' es 'False', devuelvo una lista vacía.
        return []
    
    print("Listo para hablar")                                                                                          # Imprimo un mensaje en la consola indicando que estoy listo para escuchar.

    with sr.Microphone() as source:                                                                                     # Abro el micrófono para capturar el audio.
        audio = recognizer.listen(source)                                                                               # Escucho el audio del micrófono y lo almaceno en la variable 'audio'

    try:
        texto = recognizer.recognize_google(audio, language="es-ES")                                                    # Uso el servicio de reconocimiento de voz de Google para convertir el audio en texto en español. ('texto' almacena el texto resultante de la conversión.)
        texto = texto.upper()                                                                                           # Convierto el texto a mayúsculas para normalizarlo.
        print(f"Has dicho: {texto}")                                                                                    # Imprimo el texto reconocido para que el usuario pueda verlo.

        palabras = texto.split()                                                                                        # Esta línea de código tiene como objetivo dividir el texto reconocido (texto) en una lista de palabras individuales. Por ejemplo: Si texto es "UNO DOS TRES CUATRO", entonces palabras será ['UNO', 'DOS', 'TRES', 'CUATRO']
        palabras_corregidas = [correcciones[palabra] if palabra in correcciones else palabra for palabra in palabras]   # Si la palabra está en el diccionario correcciones, reemplaza la palabra actual (palabra) con su valor correspondiente en el diccionario. Si no está en el diccionario, simplemente deja la palabra sin cambios.
        
        coordenadas = []                                                                                                # Inicializo una lista para almacenar coordenadas válidas
        es_comer = False                                                                                                # Inicializo una variable para verificar si se va a comer una pieza.(inicialmente empieza en false)

        if '*' in palabras_corregidas:                                                                                  # Verifico si el texto contiene el indicador de comer pieza ('*')
            es_comer = True                                                                                             # Indico que se va a comer una pieza, entonces la variable 'es_comer' se setea en true
            palabras_corregidas.remove('*')                                                                             # Elimino el indicador de la lista de palabras corregidas.

        if not es_comer and len(palabras_corregidas) != 2:                                                              # Verifico que la cantidad de coordenadas sea correcta si no se está comiendo una pieza
            print("Se necesitan exactamente dos coordenadas válidas para enviar.")
            return []                                                                                                   # Devuelvo una lista vacía si el número de coordenadas no es correcto.

        for palabra in palabras_corregidas:
            if len(palabra) == 2 and palabra[0] in "KBCJEFGH" and palabra[1] in "12345678":                             # Verifico si la palabra es una coordenada válida (dos caracteres, el primero en "KBCJEFGH" y el segundo en "12345678").
                coordenadas.append(palabra)                                                                             # Agrego la coordenada válida a la lista.
                print(f"La coordenada '{palabra}' es válida.")                                                          # Imprimo un mensaje si la palabra es una coordenada válida.
            else:
                print(f"La palabra '{palabra}' no es una coordenada válida.")                                           # Imprimo un mensaje si la palabra no es una coordenada válida.
        
        if es_comer:
            coordenadas.append("1")                                                                                     # Si la variable 'es_comer' es true, entonces agrego un string '1' al final del vector de coordenadas.
        else:
            coordenadas.append("0")                                                                                     # Si la variable 'es_comer' es false, entonces agrego un string '0' al final del vector de coordenadas. 
        return coordenadas                                                                                              # Devuelvo la lista de coordenadas junto con el indicador de comer pieza.
    
    except sr.UnknownValueError:                                                                                        # Manejo el error si no se puede entender el audio.
        print("No se pudo entender el audio.")                                                                          # Imprimo un mensaje para que el usuario sepa cuando no se puede entender el audio.             
        return []                                                                                                       # Devuelvo una lista vacía si no se entiende el audio.
    except sr.RequestError as e:                                                                                        # Manejo el error si hay un problema al comunicarse con el servicio de reconocimiento de voz.
        print(f"Error al comunicarse con el servicio de reconocimiento de voz: {e}")                                    # Imprimio un mensaje para que el usuario sepa que hay un error con la comunicacion.
        return []                                                                                                       # Devuelvo una lista vacía si hay un problema con el servicio de reconocimiento.

# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#

arduino = serial.Serial('COM4', 9600)                                                                                   # Creo una conexión serial con el dispositivo Arduino en el puerto COM4 a una velocidad de 9600 baudios.

def enviar_coordenadas_a_arduino(coordenadas):                                                                          # Defino la función 'enviar_coordenadas_a_arduino' que toma un parámetro llamado 'coordenadas'.
    if len(coordenadas) == 3:                                                                                           # Verifico si la lista 'coordenadas' tiene exactamente 3 elementos.
        coord_inicial = coordenadas[0].replace('J', 'D').replace('K', 'A')                                              # Reemplazo los caracteres 'J' por 'D' y 'K' por 'A' en las coordenadas iniciales 
        coord_final = coordenadas[1].replace('J', 'D').replace('K', 'A')                                                # Reemplazo los caracteres 'J' por 'D' y 'K' por 'A' en las coordenadas iniciales 
        es_comer = coordenadas[2]                                                                                       # Obtengo el indicador de si se va a comer una pieza.
        mensaje = f"{coord_inicial} {coord_final} {es_comer}\n"                                                         # Construyo el mensaje a enviar al Arduino en el formato adecuado (Tener en cuenta los espacios)
        arduino.write(mensaje.encode())                                                                                 # Escribo el mensaje en el puerto serial para enviarlo al Arduino.
        print(f"Enviando coordenadas: {mensaje}")                                                                       # Aca mando un print para saber si las coordenadas ya se enviaron al arduino.
    else:
        print("Se necesitan exactamente dos coordenadas válidas para enviar.")                                          # Imprimo un mensaje en la consola si no hay exactamente 3 elemtnos en el vector de coordenadas.                                    

# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#
# ----------------------------------------------------------------------------------------------------------------------------------------------------#

def Bucle_principal_de_voz():                                                                                           # Defino la función 'Bucle_principal_de_voz' que no toma ningún parámetro.

    global continuar                                                                                                    # Inicializo la variable 'continuar' con el valor 'True'. (Esta variable se utiliza para controlar el bucle principal)
    continuar = True 

    while continuar:                                                                                                    # Este bucle se ejecuta mientras la variable global 'continuar' sea verdadera.
        coordenadas = Reconocer_coordenadas()                                                                           # Llamo a la función 'Reconocer_coordenadas' para capturar las coordenadas desde el reconocimiento de voz.

        if coordenadas:                                                                                                 # Verifico si se obtuvieron coordenadas válidas.
            enviar_coordenadas_a_arduino(coordenadas)                                                                   # Llamo a la función 'enviar_coordenadas_a_arduino' para enviar las coordenadas al Arduino.
        if not continuar:                                                                                               # Si la variable global 'continuar' se establece en falso, salgo del bucle.
            break
        
        print("Presiona 'H' para capturar la siguiente coordenada o 'Q' para salir...")                                 # Imprimo un mensaje en la consola para indicar que se puede presionar 'H' para capturar la siguiente coordenada o 'Q' para salir.
        with keyboard.Listener(on_press=Presionar_tecla) as listener:                                                   # Espero a que el usuario presione una tecla ('H' o 'Q') usando el teclado.
            listener.join()

    print("Programa terminado.")                                                                                        # Imprimo un mensaje en la consola al finalizar el programa.

Bucle_principal_de_voz()                                                                                                # Ejecución del bucle principal

## Codigo para manejar el brazo robotico con la voz 